In [1]:
import os
import importlib
import logging
import time
importlib.reload(logging)
import framework
importlib.reload(framework)
import bert_qa
importlib.reload(bert_qa)
import infer_bert_qa
importlib.reload(infer_bert_qa)
import bert_utils
importlib.reload(bert_utils)
import pandas as pd
from framework import DataCuration, FeatureEngineering, StringProcessing
from retrieval import TaskRetrieval, FeatureEngineeringRetrieval, Retrieval

# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)
ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

Using TensorFlow backend.


Set up the task details. This notebook handles Document Retrieval for CARTA dataset.


In [2]:
DATASET = 'carta' # supports w2 and resume
TASK_CONFIG = {
    'task': 'retrieval'
}

task = TaskRetrieval(TASK_CONFIG)

Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [3]:
CARTA_DATA = [
    # should contain only a single document, since the huge doc will be split into many segments (each treated as a new doc for retrieval)
   '/Users/ahsaasbajaj/Documents/Data/QA_model/data'
]

DATASET_CONFIG = {
    'path': CARTA_DATA,
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0],
    'convert2txt': True
}

CARTA_GOLDEN = None
GOLDEN_CONFIG = None

data = DataCuration(ACCESS_TOKEN, DATASET_CONFIG, GOLDEN_CONFIG)

INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/QA_model/data
INFO:root:1 files loaded
INFO:root:Processing 1 IBOCR files to txt


In [4]:
data.dataset

{'annotated_AOI_4.pdf': <instabase.ocr.client.libs.ibocr.ParsedIBOCR at 0x14f97d128>}

In [5]:
filename = list(data.dataset.keys())[0]
print('filename to split and query: ', filename)

query = "Common Stocks"

NUM_FILES = len(data.dataset.keys())
stime = time.time()

DATA_ARGS = {
    'task': task,
    'dataset': data
}

TRAINING_ARGS = {
'model_file_or_path': "BM25Okapi"
}

fe = FeatureEngineeringRetrieval(DATA_ARGS)

corpus, doc_to_id_map = fe.split_doc(filename=filename, split_size=100)  # list of document segments


INFO:root:Total pages in file: 20
filename to split and query:  annotated_AOI_4.pdf
INFO:root:Total Corpus Size 247 docs


In [6]:
len(corpus)

247

In [7]:
tokenized_corpus = fe.tokenize_corpus(corpus)

model = Retrieval(DATA_ARGS, TRAINING_ARGS)
model.train(corpus, doc_to_id_map, tokenized_corpus)
output, scores = model.predict(query, len_results=5)

etime = time.time()
logging.info('Total time {} seconds'.format(etime - stime))

INFO:root:Scores available for 247 docs
INFO:root:Total time 1.1186749935150146 seconds


In [8]:
# output

In [9]:
idx = 1
for doc, score in zip(output, scores):
    print('Ranked Item {0} Matching score: {1:0.3f}'.format(idx, score))
    clean_output = " ".join(doc.split())
    print(clean_output, '\n')
    idx += 1

Ranked Item 1 Matching score: 1.931
In the event that the Company shall declare or pay, without consideration, any dividend on the Common Stock payable in any right to acquire Common Stock for no consideration, then the Company shall be deemed to have made a dividend payable in Common Stock in an amount of shares equal to the maximum number of shares issuable upon exercise of such rights to acquireas Common Stock. 

Ranked Item 2 Matching score: 1.764
10 determined by multiplying such Conversion Price by a fraction, the numerator of which shall be the number of shares of Common Stock Outstanding (as defined below) immediately prior to such issue plus the number of shares of Common Stock which the aggregate consideration received by the Company for the total number of Additional Shares of Common Stock so issued would purchase at such Conversion Price in effect immediately prior to such issue, and the denominator of which shall be the number of shares of Common Stock Outstanding immediat